In [12]:
#!pip install Pillow
#!pip install fpdf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math

In [54]:
#załadowanie plików do odczytu przez windows explorer
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#files = filedialog.askopenfilenames() #otwiera bezpośrednio pliki
Hist_dir = filedialog.askdirectory()  #otwiera folder z plikami

import os
from pathlib import Path

files = sorted(Path(Hist_dir).iterdir(), key=os.path.getmtime)

In [29]:
def clean(df):
    df = df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]).reset_index(drop = True)
    return df

def make_df(file):
    df = pd.read_table(file,header = None)
    df = df.drop([0,1,2,3,4,5,6,7,10,15,16]).reset_index(drop = True)
    return df

def number(Hist_dir):
    counter=0
    lista=list()
    for i in str(Hist_dir):
        if i.isdigit() and counter<4:
            lista.append(i)
            counter+=1
    return "".join(lista)   

In [55]:
#załadowane dane łączy w jeden DataFrame
df = pd.concat(list(pd.read_table(f,header = None) for f in files),axis=1)
#wyrzucenie zbędnych wierszy
df = df.drop([0,1,2,3,4,5,6,7,10,15,16]).reset_index(drop = True)
df

,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
0,X position (mm),0.000000,X position (mm),0.500000,X position (mm),1.000000,X position (mm),1.500000,X position (mm),2.000000,...,X position (mm),17.000000,X position (mm),17.500000,X position (mm),18.000000,X position (mm),18.500000,X position (mm),19.000000
1,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,...,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000
2,Remanence (mdeg),58.809817,Remanence (mdeg),61.035841,Remanence (mdeg),63.082785,Remanence (mdeg),61.555202,Remanence (mdeg),61.267077,...,Remanence (mdeg),158.461651,Remanence (mdeg),176.513440,Remanence (mdeg),180.272862,Remanence (mdeg),186.248228,Remanence (mdeg),191.983232
3,Saturation (mdeg),64.635595,Saturation (mdeg),68.691682,Saturation (mdeg),68.563272,Saturation (mdeg),67.950487,Saturation (mdeg),67.120216,...,Saturation (mdeg),159.101203,Saturation (mdeg),181.114377,Saturation (mdeg),181.514440,Saturation (mdeg),190.729694,Saturation (mdeg),196.841622
4,Coercive field (Oe),325.157351,Coercive field (Oe),320.835805,Coercive field (Oe),328.749606,Coercive field (Oe),323.002501,Coercive field (Oe),319.552638,...,Coercive field (Oe),1767.446873,Coercive field (Oe),1492.953577,Coercive field (Oe),1261.719206,Coercive field (Oe),1069.324069,Coercive field (Oe),935.755815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,-11.604400,-62.152335,-11.575000,-64.591295,-11.542600,-64.386943,-11.521700,-65.403397,-11.495300,-65.002654,...,-11.198200,-173.115711,-11.202400,-176.358811,-11.200600,-164.816879,-11.193300,-189.721338,-11.199700,-191.754246
514,-11.666300,-62.964437,-11.637400,-64.591295,-11.618400,-64.795648,-11.585200,-65.403397,-11.561300,-65.002654,...,-11.275300,-175.143312,-11.270600,-178.381104,-11.280300,-166.032378,-11.272800,-189.721338,-11.279100,-191.350849
515,-11.740000,-62.964437,-11.710600,-64.994692,-11.679700,-64.386943,-11.660000,-65.000000,-11.634700,-65.002654,...,-11.340500,-151.656051,-11.346200,-178.789809,-11.343500,-168.866773,-11.350300,-191.345541,-11.342500,-190.135350
516,-11.802600,-63.367834,-11.784100,-64.994692,-11.752500,-64.386943,-11.733800,-64.596603,-11.698700,-64.593949,...,-11.416600,-146.390658,-11.411700,-176.762208,-11.418200,-174.132166,-11.412100,-189.317941,-11.417200,-189.323248


In [4]:
def rysowanie (df):
    x_position=float(df.iloc[0,1])
    y_position=float(df.iloc[1,1])
    x = df.iloc[6:,0].astype(float)
    y = df.iloc[6:,1].astype(float)
    plt.plot(x,y)
    plt.ylabel("Kerr signal [a.u.]",size = 20)
    plt.xlabel("Magnetic field [kOe]", size = 20)
    plt.title(f"x={x_position} mm, y ={y_position} mm")
    plt.minorticks_on()
    plt.tick_params(axis="both", which = "major", length=5, direction ="in", labelsize= 15)
    plt.tick_params(axis="both", which = "minor", length=3, direction ="in")
    plt.show()
rysowanie(df)    

NameError: name 'df' is not defined

In [48]:
%%capture   
#funkcja magiczna nie pozwalająca wyświetlać outputu całej komórki. !!! ważne musi być zastosowana w pierwszej linii komórki

ncols=4
nrows=5
counter=0
Errors =list()
font=40
plt.rcParams["figure.figsize"] = (50,30)
for k in np.arange(math.ceil(len(files)/(ncols*nrows))):
    try:
    
        fig, ax = plt.subplots(nrows, ncols,constrained_layout=True)
#fig, ax = plt.subplots(nrows=2, ncols=3, sharex='col', sharey='row')        
        for i in np.arange(nrows):
            for j in np.arange(ncols):
                df= make_df(files[counter])
                ax[i,j].plot(df.iloc[6:,0].astype(float),df.iloc[6:,1].astype(float),"r")
                ax[i,j].set_title(f"x = {float(df.iloc[0,1])} nm, y = {float(df.iloc[1,1])} nm",fontsize=font)
                ax[i,j].set_xlabel("H [kOe]",fontsize=font)
                ax[i,j].set_ylabel("Kerr signal [a.u.]",fontsize=font)
                ax[i,j].minorticks_on()
                ax[i,j].tick_params(axis="both", which = "major", length=20, direction ="in",labelsize=20)
                ax[i,j].tick_params(axis="both", which = "minor", length=10, direction ="in")
                counter += 1
                
    except IndexError:
        Errors.append((i,j))
#zapis wszystkich plotow do listy. 1 element listy to wszystkie ploty dla danego k
figs = list(map(plt.figure, plt.get_fignums()))

In [60]:
#zapis do pdfu z listy figs
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('test.pdf')
#pp = PdfPages(f'{number(files)}.pdf')
for image in figs:
    pp.savefig(image)
pp.close()

In [59]:
%%capture   
#funkcja magiczna nie pozwalająca wyświetlać outputu całej komórki. !!! ważne musi być zastosowana w pierwszej linii komórki

ncols=4
nrows=5
counter=0
Errors =list()
font=40
plt.rcParams["figure.figsize"] = (50,30)
for k in np.arange(math.ceil(len(files)/(ncols*nrows))):
    try:
    
        fig, ax = plt.subplots(nrows, ncols,constrained_layout=True)
#fig, ax = plt.subplots(nrows=2, ncols=3, sharex='col', sharey='row')        
        for i in np.arange(nrows):
            for j in np.arange(ncols):
                #df= make_df(files[counter])
                ax[i,j].plot(df.iloc[6:,0+2*counter].astype(float),df.iloc[6:,1+2*counter].astype(float),"r")
                ax[i,j].set_title(f"x = {float(df.iloc[0,1])} nm, y = {float(df.iloc[1,1])} nm",fontsize=font)
                ax[i,j].set_xlabel("H [kOe]",fontsize=font)
                ax[i,j].set_ylabel("Kerr signal [a.u.]",fontsize=font)
                ax[i,j].minorticks_on()
                ax[i,j].tick_params(axis="both", which = "major", length=20, direction ="in",labelsize=20)
                ax[i,j].tick_params(axis="both", which = "minor", length=10, direction ="in")
                counter += 1
                
    except IndexError:
        Errors.append((i,j))
#zapis wszystkich plotow do listy. 1 element listy to wszystkie ploty dla danego k
figs = list(map(plt.figure, plt.get_fignums()))

In [58]:
df.head(10)

,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
0,X position (mm),0.000000,X position (mm),0.500000,X position (mm),1.000000,X position (mm),1.500000,X position (mm),2.000000,...,X position (mm),17.000000,X position (mm),17.500000,X position (mm),18.000000,X position (mm),18.500000,X position (mm),19.000000
1,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,Y position (mm),0.000000,...,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000,Y position (mm),19.000000
2,Remanence (mdeg),58.809817,Remanence (mdeg),61.035841,Remanence (mdeg),63.082785,Remanence (mdeg),61.555202,Remanence (mdeg),61.267077,...,Remanence (mdeg),158.461651,Remanence (mdeg),176.513440,Remanence (mdeg),180.272862,Remanence (mdeg),186.248228,Remanence (mdeg),191.983232
3,Saturation (mdeg),64.635595,Saturation (mdeg),68.691682,Saturation (mdeg),68.563272,Saturation (mdeg),67.950487,Saturation (mdeg),67.120216,...,Saturation (mdeg),159.101203,Saturation (mdeg),181.114377,Saturation (mdeg),181.514440,Saturation (mdeg),190.729694,Saturation (mdeg),196.841622
4,Coercive field (Oe),325.157351,Coercive field (Oe),320.835805,Coercive field (Oe),328.749606,Coercive field (Oe),323.002501,Coercive field (Oe),319.552638,...,Coercive field (Oe),1767.446873,Coercive field (Oe),1492.953577,Coercive field (Oe),1261.719206,Coercive field (Oe),1069.324069,Coercive field (Oe),935.755815
5,Field in-plane (kOe),-22.537078,Field in-plane (kOe),12.179771,Field in-plane (kOe),15.394453,Field in-plane (kOe),17.547014,Field in-plane (kOe),7.826831,...,Field in-plane (kOe),4.146458,Field in-plane (kOe),5.098980,Field in-plane (kOe),6.174759,Field in-plane (kOe),2.362321,Field in-plane (kOe),1.466834
6,-12.072500,-69.455945,-12.035000,-64.994692,-12.009100,-66.414544,-11.984300,-64.193206,-11.961200,-64.593949,...,-11.670200,-123.306794,-11.668600,-173.115711,-11.669000,-163.601380,-11.668800,-180.411359,-11.666200,-191.350849
7,-12.072300,-69.455945,-12.033400,-64.994692,-12.006800,-66.414544,-11.981700,-64.193206,-11.958700,-64.593949,...,-11.667800,-123.306794,-11.666000,-173.115711,-11.666500,-163.601380,-11.666400,-180.411359,-11.663200,-191.350849
8,-12.045900,-69.455945,-12.006500,-64.994692,-11.980100,-66.414544,-11.955200,-64.193206,-11.932000,-64.593949,...,-11.640000,-123.306794,-11.638400,-173.115711,-11.638900,-163.601380,-11.632300,-180.411359,-11.635200,-191.350849
9,-12.006400,-68.643843,-11.966800,-64.994692,-11.939100,-66.414544,-11.915700,-64.193206,-11.892900,-64.593949,...,-11.599100,-136.671975,-11.597400,-167.446921,-11.596500,-162.385881,-11.597000,-181.218153,-11.594000,-191.350849
